# Evaluation of SCA-Protected Implementations of Ascon128

- Plain SimpleSerial interface
- Shared AEAD interface
- STM32F4/STM32F3/STM32F0

### Note

- Make sure this jupyter script is located in the `jupyter` folder of your chipwhisperer installation.
- Make sure https://github.com/ascon/simpleserial-ascon is checked out in `hardware/victims/firmware` of your chipwhisperer installation:
```
cd hardware/victims/firmware
git clone https://github.com/ascon/simpleserial-ascon.git
```
- For more performance, set the baud rate to 230400 for SS_VER!=SS_VER_2_0 in e.g. `hardware/victims/firmware/hal/stm32f4/stm32f4_hal.c`:
```
UartHandle.Init.BaudRate = 230400;
```

## Setup Target Board

In [ ]:
PLATFORM = 'CW308_STM32F4'
CRYPTO_TARGET = 'NONE'
SCOPETYPE = 'OPENADC'
SS_VER = 'SS_VER_1_1'
SS_SHARED = 0
BAUD_RATE = 38400
#BAUD_RATE = 230400
DATA_LEN = 96
RESP_LEN = 96

## Detect Chipwhisperer Board

In [ ]:
%run "Setup_Scripts/Setup_Generic.ipynb"

### Setup Target Algorithm

In [ ]:
CRYPTO_PATH = 'crypto_aead/ascon128v12/protected_bi32_armv6'
NUM_SHARES_KEY = 3
NUM_SHARES_NPUB = 3
NUM_SHARES_AD = 3
NUM_SHARES_M = 3
NUM_SHARES_C = 3
ASCON_ROR_SHARES = 5
ASCON_EXTERN_BI = 0
ASCON_PA_ROUNDS = 12
ASCON_PB_ROUNDS = 6
TRACES = 10000000

In [ ]:
EXTRA_OPTS = ' -DNUM_SHARES_KEY=' + str(NUM_SHARES_KEY)
EXTRA_OPTS += ' -DNUM_SHARES_NPUB=' + str(NUM_SHARES_NPUB)
EXTRA_OPTS += ' -DNUM_SHARES_AD=' + str(NUM_SHARES_AD)
EXTRA_OPTS += ' -DNUM_SHARES_M=' + str(NUM_SHARES_M)
EXTRA_OPTS += ' -DNUM_SHARES_C=' + str(NUM_SHARES_C)
EXTRA_OPTS += ' -DASCON_ROR_SHARES=' + str(ASCON_ROR_SHARES)
EXTRA_OPTS += ' -DASCON_EXTERN_BI=' + str(ASCON_EXTERN_BI)
EXTRA_OPTS += ' -DASCON_PA_ROUNDS=' + str(ASCON_PA_ROUNDS)
EXTRA_OPTS += ' -DASCON_PB_ROUNDS=' + str(ASCON_PB_ROUNDS)
EXTRA_OPTS += ' -DSS_SHARED=' + str(SS_SHARED)
EXTRA_OPTS += ' -DDATA_LEN=' + str(DATA_LEN)
EXTRA_OPTS += ' -DRESP_LEN=' + str(RESP_LEN)

## Compile

In [ ]:
%%bash -s "$SS_VER" "$PLATFORM" "$CRYPTO_TARGET" "$CRYPTO_PATH" "$EXTRA_OPTS"
cd ../hardware/victims/firmware/simpleserial-ascon
rm -rf objdir-$2/Implementations/$4 objdir/Implementations/$4 objdir .dep
mkdir -p objdir-$2/Implementations/$4 objdir/Implementations/$4
make SS_VER=$1 PLATFORM=$2 CRYPTO_TARGET=$3 CRYPTO_PATH=$4 EXTRA_OPTS="$5"

## Flash Binary on the Victim Board

**Note:** If you use a STM32F target board of Revision-02 then you first need to set a specific jumper cable on the UFO board for the following code to work.

For more information have a look here: https://rtfm.newae.com/Targets/UFO%20Targets/CW308T-STM32F/

In [ ]:
HEXFILE = "../hardware/victims/firmware/simpleserial-ascon/simpleserial-ascon-" + PLATFORM + ".hex"
cw.program_target(scope, prog, HEXFILE)

## Communication Test

In [ ]:
target.flush()
target.baud = BAUD_RATE

print('Baudrate:')
print(target.baud)

print('Available Commands:')
print(*target.get_simpleserial_commands(),sep = "\n")

## Echo Test

In [ ]:
import time

CMD_ECHO_TEST = 'e'

data = bytearray.fromhex(''.join('{:02x}'.format(x) for x in list(range(DATA_LEN))))

target.flush()
target.simpleserial_write(CMD_ECHO_TEST,data)
time.sleep(0.1)
response = target.simpleserial_read('r', RESP_LEN, end='\n', timeout=250, ack=True)

print(list(response))

## Interface Constants

In [ ]:
import numpy as np
import random

# Command
CMD_ASCON = 'a'
CMD_ECHO_TEST = 'e'

# Flags
M = 0x01              # data contains message 
C = 0x02              # data contains ciphertext
A = 0x04              # data contains associated data
N = 0x08              # data contains nonce
K = 0x10              # data contains key
RUN_ENC = 0x20        # run enc after data transmission
RUN_DEC = 0x40        # run dec after data transmission
OMIT_RESULT = 0x80    # omit returning result after encryption/decryption

## Encryption:  `mlen = 0`, `alen = 0`

In [ ]:
# Define input data
m = bytearray.fromhex('')
a = bytearray.fromhex('')
n = bytearray.fromhex('000102030405060708090a0b0c0d0e0f')
k = bytearray.fromhex('000102030405060708090a0b0c0d0e0f')

# Send nonce and key
data = bytearray([N|K])
data += n
data += k
assert(len(data) <= DATA_LEN)
data += bytearray.fromhex('00'*(DATA_LEN-len(data)))
target.flush()
target.simpleserial_write(CMD_ASCON,data)
time.sleep(0.1)

# Send and receive data
data = bytearray([M|A|RUN_ENC])
data += bytearray([len(m)]) + m
data += bytearray([len(a)]) + a
assert(len(data) <= DATA_LEN)
data += bytearray.fromhex('00'*(DATA_LEN-len(data)))
target.flush()
target.simpleserial_write(CMD_ASCON,data)
time.sleep(0.1)
response = target.simpleserial_read('r', RESP_LEN, end='\n', timeout=250, ack=True)

# Extract and print response
c = response[0:len(m)]
t = response[len(m):len(m) + 16]
print('c: ' + ''.join('{:02x}'.format(x) for x in c[:len(m)]))
print('t: ' + ''.join('{:02x}'.format(x) for x in t))

## Encryption:  `mlen = 4`, `alen = 4`

In [ ]:
# Define input data
m = bytearray.fromhex('00010203')
a = bytearray.fromhex('00010203')
n = bytearray.fromhex('000102030405060708090a0b0c0d0e0f')
k = bytearray.fromhex('000102030405060708090a0b0c0d0e0f')

# Send nonce and key
data = bytearray([N|K])
data += n
data += k
assert(len(data) <= DATA_LEN)
data += bytearray.fromhex('00'*(DATA_LEN-len(data)))
target.flush()
target.simpleserial_write(CMD_ASCON,data)
time.sleep(0.1)

# Send and receive data
data = bytearray([M|A|RUN_ENC])
data += bytearray([len(m)]) + m
data += bytearray([len(a)]) + a
assert(len(data) <= DATA_LEN)
data += bytearray.fromhex('00'*(DATA_LEN-len(data)))
target.flush()
target.simpleserial_write(CMD_ASCON,data)
time.sleep(0.1)
response = target.simpleserial_read('r', RESP_LEN, end='\n', timeout=250, ack=True)

# Extract and print response
c = response[0:len(m)]
t = response[len(m):len(m) + 16]
print('c: ' + ''.join('{:02x}'.format(x) for x in c[:len(m)]))
print('t: ' + ''.join('{:02x}'.format(x) for x in t))

## Decryption, `clen = 0`, `alen = 0`

In [ ]:
# Define input data
c = bytearray.fromhex('e355159f292911f794cb1432a0103a8a')
a = bytearray.fromhex('')
n = bytearray.fromhex('000102030405060708090a0b0c0d0e0f')
k = bytearray.fromhex('000102030405060708090a0b0c0d0e0f')

# Send nonce and key
data = bytearray([N|K])
data += n
data += k
assert(len(data) <= DATA_LEN)
data += bytearray.fromhex('00'*(DATA_LEN-len(data)))
target.flush()
target.simpleserial_write(CMD_ASCON,data)
time.sleep(0.1)

# Send and receive data
data = bytearray([C|A|RUN_DEC])
data += bytearray([len(c)]) + c[0:-16] + c[-16:]
data += bytearray([len(a)]) + a
assert(len(data) <= DATA_LEN)
data += bytearray.fromhex('00'*(DATA_LEN-len(data)))
target.flush()
target.simpleserial_write(CMD_ASCON,data)
time.sleep(0.1)
response = target.simpleserial_read('r', RESP_LEN, end='\n', timeout=250, ack=True)

# Extract and print response
result = response[0]
m = response[1:1+len(c)-16]
print('r: ' + '{:02x}'.format(result))
print('m: ' + ''.join('{:02x}'.format(x) for x in m))

## Decryption, `clen = 4`, `alen = 4`

In [ ]:
# Define input data
c = bytearray.fromhex('7763f8ba02b1e06bc3f2370da5b314302543e9d0')
a = bytearray.fromhex('00010203')
n = bytearray.fromhex('000102030405060708090a0b0c0d0e0f')
k = bytearray.fromhex('000102030405060708090a0b0c0d0e0f')

# Send nonce and key
data = bytearray([N|K])
data += n
data += k
assert(len(data) <= DATA_LEN)
data += bytearray.fromhex('00'*(DATA_LEN-len(data)))
target.flush()
target.simpleserial_write(CMD_ASCON,data)
time.sleep(0.1)

# Send and receive data
data = bytearray([C|A|RUN_DEC])
data += bytearray([len(c)]) + c[0:-16] + c[-16:]
data += bytearray([len(a)]) + a
assert(len(data) <= DATA_LEN)
data += bytearray.fromhex('00'*(DATA_LEN-len(data)))
target.flush()
target.simpleserial_write(CMD_ASCON,data)
time.sleep(0.1)
response = target.simpleserial_read('r', RESP_LEN, end='\n', timeout=250, ack=True)

# Extract and print response
result = response[0]
m = response[1:1+len(c)-16]
print('r: ' + '{:02x}'.format(result))
print('m: ' + ''.join('{:02x}'.format(x) for x in m[:len(c)-16]))

## Sweep: Encryption/Decryption,  `mlen = 0...7`, `alen = 0...7`

In [ ]:
m = bytearray.fromhex('000102030405060708090a0b0c0d0e0f')
a = bytearray.fromhex('000102030405060708090a0b0c0d0e0f')
n = bytearray.fromhex('000102030405060708090a0b0c0d0e0f')
k = bytearray.fromhex('000102030405060708090a0b0c0d0e0f')

target.flush()

for mlen in range(8):
    for alen in range(8):
        print('mlen: {}, alen: {}'.format(mlen,alen), end='')
        #print(' m: ' + ''.join('{:02x}'.format(x) for x in m[:mlen]), end='')
        #print(' a: ' + ''.join('{:02x}'.format(x) for x in a[:alen]), end='')
        # Send key and nonce #
        data = bytearray([N|K])
        data += n
        data += k
        assert(len(data) <= DATA_LEN)
        data_pad = bytearray.fromhex('00'*DATA_LEN)
        data_pad[:len(data)] = data
        target.flush()
        target.simpleserial_write(CMD_ASCON,data_pad)
        time.sleep(0.1)
        # Encrypt #
        data = bytearray([M|A|RUN_ENC])
        data += bytearray([mlen]) + m[:mlen]
        data += bytearray([alen]) + a[:alen]
        assert(len(data) <= DATA_LEN)
        data += bytearray.fromhex('00'*(DATA_LEN-len(data)))
        target.flush()
        target.simpleserial_write(CMD_ASCON,data)
        time.sleep(0.1)
        response = target.simpleserial_read('r', RESP_LEN, end='\n', timeout=250, ack=True)
        c = response[0:mlen]
        t = response[mlen:mlen + 16]
        print(' c: ' + ''.join('{:02x}'.format(x) for x in c[:mlen]), end='')
        print(' t: ' + ''.join('{:02x}'.format(x) for x in t), end='')
        # Decrypt #
        c = c[:mlen] + t
        data = bytearray([C|A|RUN_DEC])
        data += bytearray([len(c)]) + c[0:-16] + c[-16:]
        data += bytearray([alen]) + a[:alen]
        assert(len(data) <= DATA_LEN)
        data += bytearray.fromhex('00'*(DATA_LEN-len(data)))
        target.flush()
        target.simpleserial_write(CMD_ASCON,data)
        time.sleep(0.1)
        response = target.simpleserial_read('r', RESP_LEN, end='\n', timeout=250, ack=True)
        result = response[0]
        mr = response[1:1+mlen]
        print(' r: ' + '{:02x}'.format(result), end='')
        print(' m: ' + ''.join('{:02x}'.format(x) for x in mr[:len(c)-16]), end='')
        assert(result == 0)
        print(' -> OK')

## T-Test: Encryption, `mlen = 4`, `alen = 4`

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import random
import pathlib
from tqdm import tqdm

# Measurement settings
traces = TRACES
samples_per_trace = 24400

# Variables for one-pass 1st-order t-test
n_fix = 0
n_rnd = 0
S1_fix = np.zeros((samples_per_trace,),dtype=np.double)
S1_rnd = np.zeros((samples_per_trace,),dtype=np.double)
S2_fix = np.zeros((samples_per_trace,),dtype=np.double)
S2_rnd = np.zeros((samples_per_trace,),dtype=np.double)

# Define input data
m = bytearray.fromhex('00010203')
a = bytearray.fromhex('00010203')
n = bytearray.fromhex('000102030405060708090a0b0c0d0e0f')
k = bytearray.fromhex('000102030405060708090a0b0c0d0e0f')
c = bytearray.fromhex('000102030405060708090a0b0c0d0e0f10111213')

flags = M|A|N|K|RUN_ENC|OMIT_RESULT
#flags = C|A|N|K|RUN_DEC|OMIT_RESULT

target.flush()
scope.adc.samples = samples_per_trace
scope.clock.adc_src = 'clkgen_x4'

progress = sorted(set([1000, 2000, 5000, 10000, 20000, 50000, 100000, 200000, 500000, traces]))
for i in tqdm(range(traces)):
    # Setup data #
    data = bytearray([flags])
    coinflip = random.getrandbits(1)
    if coinflip:
        # Fix input
        if flags & RUN_ENC:
            data += bytearray([len(m)])
            data += m
        if flags & RUN_DEC:
            data += bytearray([len(c)])
            data += c[0:-16]
            data += c[-16:]
        data += bytearray([len(a)])
        data += a
        data += n
    else:
        # Random input
        if flags & RUN_ENC:
            data += bytearray([len(m)]) 
            data += bytearray((random.getrandbits(8) for x in range(len(m))))
        if flags & RUN_DEC:
            data += bytearray([len(c)])
            data += bytearray((random.getrandbits(8) for x in range(len(c)-16)))
            data += bytearray((random.getrandbits(8) for x in range(16)))
        data += bytearray([len(a)])
        data += bytearray((random.getrandbits(8) for x in range(len(a))))
        data += bytearray((random.getrandbits(8) for x in range(len(n))))
    data += k
    if len(data) > DATA_LEN:
        print(len(data))
    assert(len(data) <= DATA_LEN)
    data += bytearray.fromhex('00'*(DATA_LEN-len(data)))
    # Measure trace #
    scope.arm()
    target.simpleserial_write(CMD_ASCON,data)
    scope.capture()
    trace = scope.get_last_trace()
    if i == 0:
        samples_per_trace = min(scope.adc.trig_count,samples_per_trace)
    # Process trace #
    if coinflip:
        n_fix += 1
        S1_fix += trace
        S2_fix += np.square(trace)
    else:
        n_rnd += 1
        S1_rnd += trace
        S2_rnd += np.square(trace)
    if i+1 in progress or (i+1)%1000000 == 0:
        print("samples: " + str(scope.adc.trig_count))
        # Generate t-test plot
        mu_fix = S1_fix/n_fix
        mu_rnd = S1_rnd/n_rnd
        var_fix = S2_fix/n_fix - np.square(mu_fix)
        var_rnd = S2_rnd/n_rnd - np.square(mu_rnd)
        tscore = np.divide((mu_fix - mu_rnd),np.sqrt(var_fix/n_fix + var_rnd/n_rnd))
        tscore = tscore[:samples_per_trace]
        plt.figure(figsize=(16,9))
        plt.plot(tscore)
        plt.plot([0,len(tscore)-1],[4.5,4.5],'r-')
        plt.plot([0,len(tscore)-1],[-4.5,-4.5],'r-')
        plt.title(str(n_fix+n_rnd))
        plt.xlim((0,len(tscore)))
        plt.ylim((-10,10))
        pathlib.Path("ttest/" + PLATFORM).mkdir(parents=True, exist_ok=True)
        plt.savefig("ttest/" + PLATFORM + "/{}.png".format(n_fix+n_rnd))
        plt.show()

In [ ]:
print("samples: " + str(scope.adc.trig_count))
print("cycles:  " + str(scope.adc.trig_count//4))

## Disconnect from Board

In [ ]:
scope.dis()
target.dis()